In [ ]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import numpy as np
import requests
import json
import csv

# ua=UserAgent()
# headers={"User-Agent":ua.random}
# productid = ['35216&productId=5089271','136061&productId=5089275','22778&productId=5475612','7021&productId=6784504']#四种颜色id
def commentSave(list_comment):
    file = open('JDComment_data.csv','w',newline = '')
    writer = csv.writer(file)
    writer.writerow(['用户ID','评论内容','会员级别','点赞数','回复数','得分','购买时间','手机型号'])
    for i in range(len(list_comment)):
        writer.writerow(list_comment[i])
    file.close()
    print('存入成功')

def getCommentData(maxPage):
    sig_comment = []
    global list_comment
    cur_page = 0
    while cur_page < maxPage:
        url = 'https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv%s&score=%s&sortType=5&page=%s&pageSize=10&isShadowSku=0&fold=1'%(proc,i,cur_page)
        try:
            response = requests.get(url=url, headers=headers)
            time.sleep(np.random.rand()*3)
            jsonData = response.text
            startLoc = jsonData.find('{')
            #print(jsonData[::-1])//字符串逆序
            jsonData = jsonData[startLoc:-2]
            jsonData = json.loads(jsonData)
            pageLen = len(jsonData['comments'])
            print("当前第%s页"%cur_page)
            cur_page += 1
            for j in range(0,pageLen):
                userId = jsonData['comments'][j]['id']#用户ID
                content = jsonData['comments'][j]['content']#评论内容
                levelName = jsonData['comments'][j]['userLevelName']#会员级别
                voteCount = jsonData['comments'][j]['usefulVoteCount']#点赞数
                replyCount = jsonData['comments'][j]['replyCount']#回复数目
                starStep = jsonData['comments'][j]['score']#得分
                creationTime = jsonData['comments'][j]['creationTime']#购买时间
                referenceName = jsonData['comments'][j]['referenceName']#手机型号
                sig_comment.append(userId)#每一行数据
                sig_comment.append(content)
                sig_comment.append(levelName)
                sig_comment.append(voteCount)
                sig_comment.append(replyCount)
                sig_comment.append(starStep)
                sig_comment.append(creationTime)
                sig_comment.append(referenceName)
                list_comment.append(sig_comment)
                sig_comment = []
        except:
            time.sleep(5)
            cur_page -= 1
            print('网络故障或者是网页出现了问题，五秒后重新连接')
    return list_comment

if __name__ == "__main__":
    global list_comment
    ua=UserAgent()
    headers={"User-Agent":ua.random}
    #手机四种颜色对应的产品id参数
    productid = ['35216&productId=5089271','136061&productId=5089275','22778&productId=5475612','7021&productId=6784504']
    list_comment = [[]]
    sig_comment = []
    for proc in productid:#遍历产品颜色
        i = 0
        while i < 8:#遍历排序方式
            if(i == 6):
                continue
             #先访问第0页获取最大页数，再进行循环遍历
            url = 'https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv%s&score=%s&sortType=5&page=%s&pageSize=10&isShadowSku=0&fold=1'%(proc,i,0)
            print(url)
            i += 1
            try:
                response = requests.get(url=url, headers=headers)
                jsonData = response.text
                startLoc = jsonData.find('{')
                jsonData = jsonData[startLoc:-2]
                jsonData = json.loads(jsonData)
                print("最大页数%s"%jsonData['maxPage'])
                getCommentData(jsonData['maxPage'])#遍历每一页
            except:
                i -= 1
                print("wating---")
                time.sleep(5)
                #commentSave(list_comment)
    print("爬取结束，开始存储-------")
    commentSave(list_comment)

https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv35216&productId=5089271&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&fold=1
最大页数100
当前第0页
当前第1页
当前第2页
当前第3页
当前第4页


### 数据清洗

In [19]:
#读取数据库数据
import pandas as pd
from pandas import DataFrame as df
Comment_data = pd.read_csv('JDComment_data.csv')  # 读取训练数据

In [20]:
Comment_data.head()#查看评论前几行数据

,用户ID,评论内容,会员级别,点赞数,回复数,得分,购买时间
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.165114e+10,外观非常好，手感很好，屏幕高清晰，音质很好，待机时间长，充电的话换成iPad12W充电器很快...,PLUS会员,12.0,5.0,5.0,2018/6/28 16:16
2,1.248497e+10,手机收到了，包装完整，打开后手机没有磨损，包装很好，第一次用苹果手机，心里激动，好多地方都不...,银牌会员,0.0,2.0,5.0,2019/2/25 0:19
3,1.244275e+10,苹果就是苹果！质感非常好！运行速度快！屏幕分辨率也很好！游戏也不卡！非常顺畅！64G也够用！...,金牌会员,7.0,1.0,5.0,2019/2/10 10:03
4,1.243381e+10,不错就是太大了，还是se用着舒服，但配置跟不上了，苹果产品质量没的说，视频是三年前的SE，也...,钻石会员,10.0,14.0,5.0,2019/2/2 19:53


填充缺省值以及删除第一行

In [21]:
Comment_data.drop(index=0,axis=0) #删除第一行
Comment_data.isnull().any()#查看哪些列存在空值

用户ID    True
评论内容    True
会员级别    True
点赞数     True
回复数     True
得分      True
购买时间    True
dtype: bool

In [22]:
#Comment_data[Comment_data.isnull().values==True]
for col in Comment_data:
    if col == "会员级别" or col == "评论内容":
        Comment_data[col].fillna('', inplace=True)
    elif col == "用户ID":
        Comment_data[col].fillna('00000000000', inplace=True)
    else:
        Comment_data[col].fillna(0, inplace=True)
Comment_data.isnull().any()#查看是否进行了填充

用户ID    False
评论内容    False
会员级别    False
点赞数     False
回复数     False
得分      False
购买时间    False
dtype: bool

(DataFrame筛选数据与loc用法)[https://blog.csdn.net/junbujianwpl/article/details/70473659]

In [23]:
Comment_data[Comment_data['评论内容']=="此用户未填写评价内容"] = ""#清空未评价信息
print(len(Comment_data[Comment_data['评论内容']!=""])) #统计有效评论数

21322


## 数据可视化及分析
### 会员等级编码

In [24]:
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
# define example
data = Comment_data[Comment_data['会员级别']!='']
values = array(data['会员级别'])
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
#print(label_encoder.get_params(False))
print(integer_encoded)
userLevel = label_encoder.classes_
print(label_encoder.classes_)

['PLUS会员' '银牌会员' '金牌会员' ... '金牌会员' '银牌会员' '钻石会员']
[0 7 4 ... 4 7 5]
['PLUS会员' 'PLUS会员[试用]' '企业会员' '注册会员' '金牌会员' '钻石会员' '铜牌会员' '银牌会员']


### 购买时间处理

In [25]:
hourList = []
month = []

useLessData = 0
for i in Comment_data['购买时间']:
    try:
        #print(str(i))
        hourList.append(str(i).split(' ')[1].split(':')[0])
        month.append(str(i).split(' ')[0].split('/')[1])
    except:
        useLessData += 1
print('无效数据有%s条'%useLessData)

无效数据有609条


In [ ]:
#from numpy import DataFrame
import numpy as np
v = [[]]
for level in label_encoder.classes_:
    print(level)
    df = Comment_data[Comment_data['会员级别'] == level]
    hourList = []
    month = []
    for i in df['购买时间']:
        try:
            #print(str(i))
            month.append(str(i).split(' ')[0].split('/')[1])
        except:
            useLessData += 1
    data = [month.count(str(_)) for _ in range(0, 12)]
    #print(data)
    v.append(data)
print(v)

In [32]:
import random
from pyecharts import Bar
attr = ["{}时".format(i) for i in range(24)]
v1 = [hourList.count(str(_)) for _ in range(24)]
bar = Bar("Bar - time - sale",title_color='#FF0000',background_color='#7EC0EE')
bar.add("", attr, v1, is_label_show=True, is_datazoom_show=True)
bar

分析一天中不同时段消费者的手机购买情况，可以分析得出，大部分消费者在午时（10-12时）和晚上（20-22时）出现了消费高峰期，在此时段顾客购买商品的概率更大，他们浏览商品的机会更多

In [1]:
from pyecharts import Line

attr = ["一月", "二月", "三月", "四月", "五月", "六月",'七月','八月','九月','十月','十一月','十二月']
#plus = Comment_data[Comment_data['会员级别']=='PLUS会员']
v1 = [month.count(str(_)) for _ in range(0, 12)]
#v2 = [55, 60, 16, 20, 15, 80]
line = Line("折线图示例",title_color='#20B2AA',background_color='#F0F8FF')
count = 1
for level in label_encoder.classes_:
    line.add(level, attr, v[count], mark_point=["average"])
    count += 1
#line.add("PLUS会员", attr, v1, mark_point=["average"])

#line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

SyntaxError: invalid syntax (<ipython-input-1-31aff8c4d7c1>, line 9)

PLUS会员
PLUS会员[试用]
企业会员
注册会员
金牌会员
钻石会员
铜牌会员
银牌会员
[[], [0, 850, 1225, 559, 262, 300, 420, 260, 298, 232, 378, 1434], [0, 36, 183, 4, 4, 4, 0, 4, 0, 8, 0, 16], [0, 76, 24, 0, 0, 4, 0, 4, 0, 0, 8, 4], [0, 400, 636, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 599, 744, 452, 228, 232, 347, 211, 199, 203, 238, 935], [0, 420, 668, 256, 140, 168, 291, 144, 115, 116, 275, 804], [0, 323, 604, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 720, 980, 284, 171, 135, 304, 236, 211, 159, 124, 391]]
